# Praktikum 1
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

## Preprocessing

In [ ]:
train_df = pd.read_csv('train.csv')
train_df

In [ ]:
test_df = pd.read_csv('test.csv')
test_df

In [ ]:
targets_train = train_df['label']
features_train = train_df.drop(columns=['label', 'Unnamed: 0', 'Unnamed: 0.1', 'people'])

targets_test = test_df['label']
features_test = test_df.drop(columns=['label', 'Unnamed: 0', 'Unnamed: 0.1', 'people'])

print (features_train.shape)
print (targets_train.shape)
print (features_test.shape)
print (targets_test.shape)

In [ ]:
def normalize(data) :
    min_data = data.min()
    max_data = data.max()
    result = list(map(lambda x: (x-min_data)/(max_data-min_data)*1, data))
    return result

normalize_features = ['freq', 'phase', 'rssi']

for normalize_feature in normalize_features :
    features_train[normalize_feature] = normalize(features_train[normalize_feature])
    features_test[normalize_feature] = normalize(features_test[normalize_feature])
    
features_train['id'] = pd.Categorical(features_train['id'])
dfId = pd.get_dummies(features_train['id'], prefix = 'id')
features_train = pd.concat([features_train, dfId], axis=1)
features_train = features_train.drop(columns=['id'], axis=1)

features_test['id'] = pd.Categorical(features_test['id'])
dfId = pd.get_dummies(features_test['id'], prefix = 'id')
features_test = pd.concat([features_test, dfId], axis=1)
features_test = features_test.drop(columns=['id'], axis=1)

print (features_train.shape)
print (features_test.shape)

In [ ]:
features_train = np.array(features_train)
features_test = np.array(features_test)
features_train = features_train.reshape(features_train.shape[0], features_train.shape[1], 1)
features_test = features_test.reshape(features_test.shape[0], features_test.shape[1], 1)

print (features_train.shape)
print (features_test.shape)

## Model

### RNN

In [ ]:
rnn = Sequential()

### LSTM

In [ ]:
lstm = Sequential()

### GRU

In [ ]:
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation, Flatten
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
gru = Sequential()
gru.add(GRU(50, input_shape = (features_train.shape[1], 1), return_sequences = True))
gru.add(Dropout(0.2))
gru.add(GRU(1, return_sequences = False))
gru.add(Dropout(0.2))
gru.add(Activation('sigmoid'))

gru.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

gru.fit(features_train, targets_train, batch_size = 100, epochs = 10, verbose = 0)

scores = gru.evaluate(features_test, targets_test, verbose=0)

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
f1_score_test = f1_score(y_true, y_pred, average='macro')
accuracy_score_test = accuracy_score(y_true, y_pred)
confusion_matrix_test = confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
np.set_printoptions(precision=2)
print('Confusion matrix')
print(cm)
plt.figure()
plot_confusion_matrix(confusion_matrix_test)